In [13]:
import sqlalchemy as sql
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy import Column, Integer, String, BigInteger, VARCHAR, Float, ForeignKey
import datetime 
import time
import os

pd.set_option('display.max_rows', 100)

In [14]:
    ss_data_cols = """Name
    Team
    Opponent
    Position
    Projection
    Actual
    Price
    Value
    proj_own
    dk_point
    dk_25_percentile
    dk_50_percentile
    dk_75_percentile
    dk_85_percentile
    dk_95_percentile
    dk_99_percentile
    fd_points
    fd_25_percentile
    fd_50_percentile
    fd_75_percentile
    fd_85_percentile
    fd_95_percentile
    fd_99_percentile
    fdraft_points
    fdraft_25_percentile
    fdraft_50_percentile
    fdraft_75_percentile
    fdraft_85_percentile
    fdraft_95_percentile
    fdraft_99_percentile
    yahoo_points
    yahoo_25_percentile
    yahoo_50_percentile
    yahoo_75_percentile
    yahoo_85_percentile
    yahoo_95_percentile
    yahoo_99_percentile
    dk_std
    fd_std
    fdraft_std
    yahoo_std
    pass_attempts
    completions
    passing_yards
    passing_td
    receptions
    receiving_yards
    receiving_td
    rushes
    rushing_yards
    rushing_td""".split('\n')
    
    _4f4_proj_cols = """Season
    Week
    PID
    Player
    Pos
    Team
    Opp
    aFPA
    aFPA_Rk
    FFPts
    Comp
    Pass_Att
    Pass_Yds
    Pass_TD
    INT
    Rush_Att
    Rush_Yds
    Rush_TD
    Rec
    Rec_Yds
    Rec_TD
    Pa1D
    Ru1D
    Rec1D
    Fum
    XP
    FG
    Grade""".split('\n')
    
    _4f4_ceil_cols = """Season
    Week
    PID
    Player
    Pos
    Team
    Opp
    aFPA
    aFPA_Rk
    FFPts
    Comp
    Pass_Att
    Pass_Yds
    Pass_TD
    INT
    Rush_Att
    Rush_Yds
    Rush_TD
    Rec
    Rec_Yds
    Rec_TD
    Pa1D
    Ru1D
    Rec1D
    Fum
    XP
    FG
    Grade
    DK_Proj
    DK_Price
    DK_Val
    DK_Pts_per_1k
    DK_Flr
    DK_Flr_Val
    DK_Flr_per_1k
    DK_Ceil
    DK_Ceil_Val
    DK_Ceil_per_1k
    DK_Wk_Change
    DK_Own_Percent
    FD_Proj
    FD_Price
    FD_Val
    FD_Pts_pr_1k
    FD_Flr
    FD_Flr_Val
    FD_Flr_pr_1k
    FD_Ceil
    FD_Ceil_Val
    FD_Ceil_pr_1k
    FD_Wk_Change
    FD_Own_Percent
    Y_Proj
    Y_Price
    Y_Val
    Y_Pt_pr_Dollar
    Y_Flr
    Y_Flr_Val
    Y_Flr_pr_Dollar
    Y_Ceil
    Y_Ceil_Val
    Y_Ceil_pr_Dollar
    Y_Wk_Change
    FyD_Proj
    FyD_Price
    FyD_Val
    FyD_Pt_pr_1k
    FyD_Flr
    FyD_Flr_Val
    FyD_Flr_pr_1k
    FyD_Ceil
    FyD_Ceil_Val
    FyD_Ceil_pr_1k
    FyD_Wk_Change
    O_U
    Line
    Team_O_U""".split('\n')
    
_4f4_wrcb_cols = ['Player', 'Team', 'Opponent', 'Matchup', 'Shadow']
_4f4_wr_fp_cols = ['Player', 'Pos', 'Team', 'G', 'W5', 'W6', 'W7', 'W8', 'DK_Pts','DK_Pts_pr_G']
_4f4_rb_fp_cols = ['Player', 'Pos', 'Team', 'G', 'W13', 'W14', 'W15', 'DK_Pts','DK_Pts_pr_G']
_4f4_rb_tar_cols = ['Player', 'Pos', 'Team', 'G', 'W13', 'W14', 'W15', 'TGTs', 'TGTs_pr_G', 'Array']
airy_wr_cols = ['full_name', 'position', 'team', 'targets', 'rec', 'rec_yards',
                   'air_yards', 'yac', 'td', 'adot', 'racr', 'ms_air', 'tgt_share', 'wopr',
                   'ppr', 'Tar_pr_Gam', 'AY_pr_Gam']
airy_te_cols = ['full_name', 'position', 'team', 'targets', 'rec', 'rec_yards',
                   'air_yards', 'yac', 'td', 'adot', 'racr', 'ms_air', 'tgt_share', 'wopr',
                   'ppr', 'Tar_pr_3']

_4f4_redZ_cols = ['Season', 'Week', 'PID', 'Player', 'Pos', 'Team', 'Opp', 'aFPA',
       'aFPA_Rk', 'FFPts', 'Comp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'INT',
       'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec', 'Rec_Yds', 'Rec_TD', 'Pa1D',
       'Ru1D', 'Rec1D', 'Fum', 'XP', 'FG', 'Grade', 'DK_proj', 'DK_price',
       'DK_val', 'DK_Pt_pr_1k', 'DK_Flr', 'DK_Flr_Val', 'DK_Flr_pr_1k',
       'DK_Ceil', 'DK_Ceil_Val', 'DK_Ceil_pr_1k', 'DK_Wk_Change',
       'DK_Own_Percent', 'FD_proj', 'FD_price', 'FD_val', 'FD_Pt_pr_1k',
       'FD_Flr', 'FD_Flr_Val', 'FD_Flr_pr_1k', 'FD_Ceil',
       'FD_Ceil_Val', 'FD_Ceil_pr_1k', 'FD_Wk_Change', 'FD_Own_Percent',
       'Y_proj', 'Y_price', 'Y_val', 'Y_Pt_pr_DOllar', 'Y_Flr',
       'Y_Flr_Val', 'Y_Flr_pr_Dollar', 'Y_Ceil', 'Y_Ceil_Val',
       'Y_Ceil_pr_Dollar', 'Y_Wk_Change', 'FyD_proj', 'FyD_price', 'FyD_val',
       'FyD_Pt_pr_1k', 'FyD_Flr', 'FyD_Flr_Val', 'FyD_Flr_pr_1k',
       'FyD_Ceil', 'FyD_Ceil_Val', 'FyD_Ceil_pr_1k', 'FyD_Wk_Change',
       'O_U', 'Line', 'Team_O_U']

etr_ceiling_cols = ['Season', 'Week', 'PID', 'Player', 'Pos', 'Team', 'Opp', 'aFPA',
       'aFPA_Rk', 'FFPts', 'Comp', 'Pass_Att', 'Pass_Yds', 'Pass_TD', 'INT',
       'Rush_Att', 'Rush_Yds', 'Rush_TD', 'Rec', 'Rec_Yds', 'Rec_TD', 'Pa1D',
       'Ru1D', 'Rec1D', 'Fum', 'XP', 'FG', 'Grade', 'DK_proj', 'DK_price',
       'DK_val', 'DK_Pt_pr_1k', 'DK_Flr', 'DK_Flr_Val', 'DK_Flr_pr_1k',
       'DK_Ceil', 'DK_Ceil_Val', 'DK_Ceil_pr_1k', 'DK_Wk_Change',
       'DK_Own_Percent', 'FD_proj', 'FD_price', 'FD_val', 'FD_Pt_pr_1k',
       'FD_Flr', 'FD_Flr_Val', 'FD_Flr_pr_1k', 'FD_Ceil',
       'FD_Ceil_Val', 'FD_Ceil_pr_1k', 'FD_Wk_Change', 'FD_Own_Percent',
       'Y_proj', 'Y_price', 'Y_val', 'Y_Pt_pr_Dollar', 'Y_Flr',
       'Y_Flr_Val', 'Y_Flr_pr_Dollar', 'Y_Ceil', 'Y_Ceil_Val',
       'Y_Ceil_pr_Dollar', 'Y_Wk_Change', 'FyD_proj', 'FyD_price', 'FyD_val',
       'FyD_Pt_pr_1k', 'FyD_Flr', 'FyD_Flr_Val', 'FyD_Flr_pr_1k',
       'FyD_Ceil', 'FyD_Ceil_Val', 'FyD_Ceil_pr_1k', 'FyD_Wk_Change',
       'O_U', 'Line', 'Team_O_U']

In [15]:
SQLALCHEMY_DATABASE_URL = f"sqlite+pysqlite:///DFS/Resources/DFS.db"

engine = create_engine(SQLALCHEMY_DATABASE_URL, pool_pre_ping=True, connect_args={"check_same_thread": True})

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
db_session = scoped_session(SessionLocal)
Base = declarative_base(bind=engine)

Base.metadata.create_all(engine)

In [16]:
class DictMixIn:
    def to_dict(self):
        return {
            column.name: getattr(self, column.name)
            if not isinstance(getattr(self, column.name), datetime.datetime)
            else getattr(self, column.name).isoformat()
            for column in self.__table__.columns
        }
    
class Players(Base, DictMixIn):
    __tablename__ = 'players'
    PID = Column(String(25), primary_key=True)
    Name = Column(String(25))
    Position = Column(String)
    Team = Column(String)
    
class SS_Data(Base, DictMixIn):
    __tablename__ = 'Saber_Sim_Data'
#     PID = Column(String, ForeignKey('players.PID'))
    Name = Column(String(30), primary_key=True)
    Team = Column(String(10), primary_key=True)
    Opponent = Column(String(10), primary_key=True)
    Position = Column(String(5), primary_key=True)
    Projection = Column(Float, primary_key=True)
    Actual = Column(String)
    Price = Column(BigInteger)
    Value = Column(Float)
#     ## added by me to keep track of week/Season
#     Date_Added = Column(Date) 
#     Week = Column(Integer)
#     Season = Column(Integer)
#     ## end of added data points
    proj_own = Column(Float)
    dk_point = Column(Float)
    dk_25_percentile = Column(Float)
    dk_50_percentile  = Column(Float)
    dk_75_percentile = Column(Float) 
    dk_85_percentile = Column(Float)
    dk_95_percentile = Column(Float) 
    dk_99_percentile = Column(Float)  
    fd_points = Column(Float) 
    fd_25_percentile = Column(Float)
    fd_50_percentile = Column(Float) 
    fd_75_percentile = Column(Float) 
    fd_85_percentile = Column(Float)
    fd_95_percentile  = Column(Float)
    fd_99_percentile = Column(Float) 
    fdraft_points = Column(Float)
    fdraft_25_percentile = Column(Float) 
    fdraft_50_percentile = Column(Float) 
    fdraft_75_percentile = Column(Float)
    fdraft_85_percentile = Column(Float) 
    fdraft_95_percentile = Column(Float) 
    fdraft_99_percentile = Column(Float)
    yahoo_points = Column(Float)
    yahoo_25_percentile = Column(Float) 
    yahoo_50_percentile = Column(Float)
    yahoo_75_percentile = Column(Float) 
    yahoo_85_percentile = Column(Float) 
    yahoo_95_percentile = Column(Float)
    yahoo_99_percentile = Column(Float)
    dk_std = Column(Float)
    fd_std = Column(Float)
    fdraft_std = Column(Float)
    yahoo_std = Column(Float)
    pass_attempts = Column(Float)
    completions = Column(Float)
    passing_yards = Column(Float)
    passing_td = Column(Float)
    receptions = Column(Float)
    receiving_yards = Column(Float)
    receiving_td = Column(Float)
    rushes = Column(Float)
    rushing_yards = Column(Float)
    rushing_td = Column(Float)
    
class _4f4_Proj(Base, DictMixIn):
    __tablename__ = '4for4_Projections'
    Season = Column(Integer, primary_key=True)
    Week = Column(Integer, primary_key=True)
    PID = Column(String(20), primary_key=True)
    Player = Column(String(20), primary_key=True)
    Pos = Column(String(20), primary_key=True)
    Team = Column(String(20), primary_key=True)
    Opp = Column(String(20))
    aFPA = Column(Float)
    aFPA_Rk = Column(Integer)
    FFPts = Column(Float)
    Comp = Column(Float)
    Pass_Att = Column(Float)
    Pass_Yds = Column(Float)
    Pass_TD = Column(Float)
    INT = Column(Float)
    Rush_Att = Column(Float)
    Rush_Yds = Column(Float)
    Rush_TD = Column(Float)
    Rec = Column(Float)
    Rec_Yds = Column(Float)
    Rec_TD = Column(Float)
    Pa1D = Column(Float)
    Ru1D = Column(Float)
    Rec1D = Column(Float)
    Fum = Column(Float)
    XP = Column(Float)
    FG = Column(Float)
    Grade = Column(String)
    
class _4f4_Ceiling(Base, DictMixIn):
    __tablename__ = '4for4_Ceiling'
    Season = Column(Integer, primary_key=True)
    Week = Column(Integer, primary_key=True)
    PID = Column(Integer, primary_key=True)
    Player = Column(String(30), primary_key=True)
    Pos = Column(String(30), primary_key=True)
    Team = Column(String(30), primary_key=True)
    Opp = Column(String(30))
    aFPA = Column(Float)
    aFPA_Rk = Column(Integer)
    FFPts = Column(Float)
    Comp = Column(Float)
    Pass_Att = Column(Float)
    Pass_Yds = Column(Float)
    Pass_TD = Column(Float)
    INT = Column(Float)
    Rush_Att = Column(Float)
    Rush_Yds = Column(Float)
    Rush_TD = Column(Float)
    Rec = Column(Float)
    Rec_Yds = Column(Float)
    Rec_TD = Column(Float)
    Pa1D = Column(Float)
    Ru1D = Column(Float)
    Rec1D = Column(Float)
    Fum = Column(Float)
    XP = Column(Float)
    FG = Column(Float)
    Grade = Column(Integer)
    DK_Proj = Column(Float)
    DK_Price = Column(Float)
    DK_Val = Column(Float)
    DK_Pts_per_1k = Column(Float)
    DK_Flr = Column(Float)
    DK_Flr_Val = Column(Float)
    DK_Flr_per_1k = Column(Float)
    DK_Ceil = Column(Float)
    DK_Ceil_Val = Column(Float)
    DK_Ceil_per_1k = Column(Float)
    DK_Wk_Change = Column(Integer)
    DK_Own_Percent = Column(Integer)
    FD_Proj = Column(Float)
    FD_Price = Column(Float)
    FD_Val = Column(Float)
    FD_Pts_pr_1k = Column(Float)
    FD_Flr = Column(Float)
    FD_Flr_Val = Column(Float)
    FD_Flr_pr_1k = Column(Float)
    FD_Ceil = Column(Float)
    FD_Ceil_Val = Column(Float)
    FD_Ceil_pr_1k = Column(Float)
    FD_Wk_Change = Column(String)
    FD_Own_Percent = Column(String)
    Y_Proj = Column(Float)
    Y_Price = Column(Float)
    Y_Val = Column(Float)
    Y_Pt_pr_Dollar = Column(Float)
    Y_Flr = Column(Float)
    Y_Flr_Val = Column(Float)
    Y_Flr_pr_Dollar = Column(Float)
    Y_Ceil = Column(Float)
    Y_Ceil_Val = Column(Float)
    Y_Ceil_pr_Dollar = Column(Float)
    Y_Wk_Change = Column(Integer)
    FyD_Proj = Column(Float)
    FyD_Price = Column(Float)
    FyD_Val = Column(Float)
    FyD_Pt_pr_1k = Column(Float)
    FyD_Flr = Column(Float)
    FyD_Flr_Val = Column(Float)
    FyD_Flr_pr_1k = Column(Float)
    FyD_Ceil = Column(Float)
    FyD_Ceil_Val = Column(Float)
    FyD_Ceil_pr_1k = Column(Float)
    FyD_Wk_Change = Column(String)
    O_U = Column(Float)
    Line = Column(Float)
    Team_O_U = Column(Float)

    
class _4f4_WR_cb(Base, DictMixIn):
    __tablename__ = '4for4_WRCB'
    Player = Column(String, primary_key=True)
    Team = Column(String, primary_key=True)
    Opponent = Column(String, primary_key=True)
    Matchup = Column(String)
    Shadow = Column(String)

class _4f4_RedZ(Base, DictMixIn):
    __tablename__ = '4f4_RedZone_Data'
    Season = Column(Integer, primary_key=True) 
    Week = Column(Float, primary_key=True) 
    PID = Column(String, primary_key=True) 
    Player = Column(String, primary_key=True) 
    Pos = Column(String, primary_key=True) 
    Team = Column(String) 
    Opp = Column(String) 
    aFPA = Column(Float) 
    aFPA_Rk = Column(Float) 
    FFPts = Column(Float) 
    Comp = Column(Float) 
    Pass_Att = Column(Float) 
    Pass_Yds = Column(Float) 
    Pass_TD = Column(Float) 
    INT = Column(Float) 
    Rush_Att = Column(Float) 
    Rush_Yds = Column(Float) 
    Rush_TD = Column(Float) 
    Rec = Column(Float) 
    Rec_Yds = Column(Float) 
    Rec_TD = Column(Float) 
    Pa1D = Column(Float) 
    Ru1D = Column(Float) 
    Rec1D = Column(Float) 
    Fum = Column(Float) 
    XP = Column(Float) 
    FG = Column(Float) 
    Grade = Column(String) 
    DK_proj = Column(Float) 
    DK_price = Column(Float) 
    DK_val = Column(Float) 
    DK_PT_pr_1k = Column(Float) 
    DK_Flr = Column(Float) 
    DK_Flr_Val = Column(Float) 
    DK_Flr_pr_1k = Column(Float) 
    DK_Ceil = Column(Float) 
    DK_Ceil_Val = Column(Float) 
    DK_Ceil_pr_1k = Column(Float) 
    DK_Wk_Change = Column(Integer) 
    DK_own_percent = Column(Integer) 
    FD_proj = Column(Float) 
    FD_price = Column(Float) 
    FD_val = Column(Float) 
    FD_PT_pr_1k = Column(Float) 
    FD_Flr = Column(Float) 
    FD_Flr_Val = Column(Float) 
    FD_Flr_pr_1k = Column(Float) 
    FD_Ceil = Column(Float) 
    FD_Ceil_Val = Column(Float) 
    FD_Ceil_pr_1k = Column(Float) 
    FD_Wk_Change = Column(Integer) 
    FD_own_percent = Column(Integer) 
    Y_proj = Column(Float) 
    Y_price = Column(Float) 
    Y_val = Column(Float) 
    Y_Pt_pr_Dollar = Column(Float) 
    Y_Flr = Column(Float) 
    Y_Flr_Val = Column(Float) 
    Y_Flr_pr_Dollar = Column(Float) 
    Y_Ceil = Column(Float) 
    Y_Ceil_Val = Column(Float) 
    Y_Ceil_pr_Dollar = Column(Float) 
    Y_Wk_Change = Column(Integer) 
    FyD_proj = Column(Float) 
    FyD_price = Column(Float) 
    FyD_val = Column(Float) 
    FyD_PT_pr_1k = Column(Float) 
    FyD_Flr = Column(Float) 
    FyD_Flr_Val = Column(Float) 
    FyD_Flr_pr_1k = Column(Float) 
    FyD_Ceil = Column(Float) 
    FyD_Ceil_Val = Column(Float) 
    FyD_Ceil_pr_1k = Column(Float) 
    FyD_Wk_Change = Column(Integer)
    O_U = Column(Float) 
    Line = Column(Float) 
    Team_O_U = Column(Float) 

class _4f4_WR_fp(Base, DictMixIn):
    __tablename__ = '4for4_WR_fantasy_points'
    Player = Column(String, primary_key=True) 
    Pos = Column(String, primary_key=True) 
    Team = Column(String, primary_key=True) 
    G = Column(Integer) 
    W5 = Column(Float) 
    W6 = Column(Float) 
    W7 = Column(Float) 
    W8 = Column(Float) 
    DK_Pts = Column(Float) 
    DK_Pts_pr_G = Column(Float) 

class _4f4_RB_fp(Base, DictMixIn):
    __tablename__ = '4for4_RB_fantasy_points'
    Player = Column(String, primary_key=True) 
    Pos = Column(String, primary_key=True) 
    Team = Column(String, primary_key=True) 
    G = Column(Integer) 
    W13 = Column(Float) 
    W14 = Column(Float) 
    W15 = Column(Float) 
    DK_Pts = Column(Float) 
    DK_Pts_pr_G = Column(Float) 

class _4f4_RB_tar(Base, DictMixIn):
    __tablename__ = '4for4_RB_target'
    Player = Column(String, primary_key=True)
    Pos = Column(String, primary_key=True) 
    Team = Column(String, primary_key=True) 
    G = Column(Integer) 
    W13 = Column(Float) 
    W14 = Column(Float) 
    W15 = Column(Float) 
    TGTs = Column(Integer) 
    TGTs_pr_G = Column(Float) 
    Array = Column(Float) 

class AirY_WR(Base, DictMixIn):
    __tablename__ = 'AirY_WR'
    full_name = Column(String, primary_key=True)
    position = Column(String, primary_key=True) 
    team = Column(String, primary_key=True) 
    targets = Column(Float, primary_key=True) 
    rec = Column(Float) 
    rec_yards = Column(Float) 
    air_yards = Column(Float) 
    yac = Column(Float) 
    td = Column(Float) 
    adot = Column(Float) 
    racr = Column(Float) 
    ms_air = Column(Float) 
    tgt_share = Column(Float) 
    wopr = Column(Float) 
    ppr = Column(Float) 
    Tar_pr_Gam = Column(Float) 
    AY_pr_Gam = Column(Float) 

class AirY_TE(Base, DictMixIn):
    __tablename__ = 'AirY_TE'
    full_name = Column(String, primary_key=True)
    position = Column(String, primary_key=True) 
    team = Column(String, primary_key=True) 
    targets = Column(Float) 
    rec = Column(Float) 
    rec_yards = Column(Float) 
    air_yards = Column(Float) 
    yac = Column(Float) 
    td = Column(Float) 
    adot = Column(Float) 
    racr = Column(Float) 
    ms_air = Column(Float) 
    tgt_share = Column(Float) 
    wopr = Column(Float) 
    ppr = Column(Float) 
    Tar_pr_3 = Column(Float) 

class ETR_Ceil(Base, DictMixIn):
    __tablename__ = 'ETR_Ceiling'
    Season = Column(Integer, primary_key=True) 
    Week = Column(Integer, primary_key=True) 
    PID = Column(String, primary_key=True) 
    Player = Column(String, primary_key=True) 
    Pos = Column(String, primary_key=True) 
    Team = Column(String) 
    Opp = Column(String) 
    aFPA = Column(Float) 
    aFPA_Rk = Column(Integer) 
    FFPts = Column(Float) 
    Comp = Column(Float) 
    Pass_Att = Column(Float) 
    Pass_Yds = Column(Float) 
    Pass_TD = Column(Float) 
    INT = Column(Float) 
    Rush_Att = Column(Float) 
    Rush_Yds = Column(Float) 
    Rush_TD = Column(Float) 
    Rec = Column(Float) 
    Rec_Yds = Column(Float) 
    Rec_TD = Column(Float) 
    Pa1D = Column(Float) 
    Ru1D = Column(Float) 
    Rec1D = Column(Float) 
    Fum = Column(Float) 
    XP = Column(Float) 
    FG = Column(Float) 
    Grade = Column(String) 
    DK_proj = Column(Float) 
    DK_price = Column(Float) 
    DK_val = Column(Float) 
    DK_PT_pr_1k = Column(Float) 
    DK_Flr = Column(Float) 
    DK_Flr_Val = Column(Float) 
    DK_Flr_pr_1k = Column(Float) 
    DK_Ceil = Column(Float) 
    DK_Ceil_Val = Column(Float) 
    DK_Ceil_pr_1k = Column(Float) 
    DK_Wk_Change = Column(Integer) 
    DK_own_percent = Column(Integer) 
    FD_proj = Column(Float) 
    FD_price = Column(Float) 
    FD_val = Column(Float) 
    FD_PT_pr_1k = Column(Float) 
    FD_Flr = Column(Float) 
    FD_Flr_Val = Column(Float) 
    FD_Flr_pr_1k = Column(Float) 
    FD_Ceil = Column(Float) 
    FD_Ceil_Val = Column(Float) 
    FD_Ceil_pr_1k = Column(Float) 
    FD_Wk_Change = Column(Integer) 
    FD_own_percent = Column(Integer) 
    Y_proj = Column(Float) 
    Y_price = Column(Float) 
    Y_val = Column(Float) 
    Y_Pt_pr_Dollar = Column(Float) 
    Y_Flr = Column(Float) 
    Y_Flr_Val = Column(Float) 
    Y_Flr_pr_Dollar = Column(Float) 
    Y_Ceil = Column(Float) 
    Y_Ceil_Val = Column(Float) 
    Y_Ceil_pr_Dollar = Column(Float) 
    Y_Wk_Change = Column(Integer) 
    FyD_proj = Column(Float) 
    FyD_price = Column(Float) 
    FyD_val = Column(Float) 
    FyD_PT_pr_1k = Column(Float) 
    FyD_Flr = Column(Float) 
    FyD_Flr_Val = Column(Float) 
    FyD_Flr_pr_1k = Column(Float) 
    FyD_Ceil = Column(Float) 
    FyD_Ceil_Val = Column(Float) 
    FyD_Ceil_pr_1k = Column(Float) 
    FyD_Wk_Change = Column(Integer)
    O_U = Column(Float) 
    Line = Column(Float) 
    Team_O_U = Column(Float) 

In [17]:
workbook_path = os.path.join('DFS', 'Resources', 'DraftKingsNoNotes.xlsx')

workbook = pd.ExcelFile(workbook_path)

ss_data = workbook.parse('SS Data')
_4f4_proj = workbook.parse('4f4 Proj Data')
_4f4_ceil = workbook.parse('4f4 Ceiling Data')
_4f4_wrcb= workbook.parse('4f4 WRCB')
_4f4_redZ = workbook.parse('4f4 Red Z Data')
_4f4_wr_fp = workbook.parse('4f4 WR FP Last 4 ')
_4f4_rb_fp = workbook.parse('4f4 RB FP Last 3')
_4f4_rb_tar = workbook.parse('4f4 RB TAR Last 3')
airy_wr = workbook.parse('AirY WR Last 4').drop(['Unnamed: 0'], axis=1)
airy_te = workbook.parse('AirY TE Last 3').drop(['Unnamed: 0'], axis=1)
etr_ceiling = workbook.parse('ETR Ceiling Data')

In [18]:
ss_data.columns = ss_data_cols
_4f4_proj.columns = _4f4_proj_cols
_4f4_ceil.columns = _4f4_ceil_cols
_4f4_wrcb.columns = _4f4_wrcb_cols
_4f4_wr_fp.columns = _4f4_wr_fp_cols
_4f4_rb_fp.columns = _4f4_rb_fp_cols
_4f4_rb_tar.columns = _4f4_rb_tar_cols
airy_wr.columns = airy_wr_cols 
airy_te.columns = airy_te_cols 

_4f4_redZ.columns = _4f4_redZ_cols
etr_ceiling.columns = etr_ceiling_cols


In [19]:
ss_data.to_sql(name='Saber_Sim_Data',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)

_4f4_proj.to_sql(name='4for4_Projections',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
_4f4_ceil.to_sql(name='4for4_Ceiling',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
_4f4_wrcb.to_sql(name='4for4_wrcb',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
_4f4_redZ.to_sql(name='4f4_RedZone_Data',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
_4f4_wr_fp.to_sql(name='4for4_WR_fantasy_points',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
_4f4_rb_fp.to_sql(name='4for4_RB_fantasy_points',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
_4f4_rb_tar.to_sql('4for4_RB_target',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
airy_wr.to_sql(name='AirY_WR',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
airy_te.to_sql(name='AirY_TE',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)
etr_ceiling.to_sql(name='ETR_Ceiling',
                con=engine,
                if_exists='replace',
                index=True,
                chunksize=200)

In [20]:
db_session.query(AirY_TE).all()[0].to_dict()

{'full_name': 'Tyler Higbee',
 'position': 'TE',
 'team': 'LA',
 'targets': 33.0,
 'rec': 26.0,
 'rec_yards': 334.0,
 'air_yards': 257.0,
 'yac': 168.0,
 'td': 1.0,
 'adot': 7.8,
 'racr': 1.3,
 'ms_air': 0.29,
 'tgt_share': 0.26,
 'wopr': 0.59,
 'ppr': 65.4,
 'Tar_pr_3': 11.0}

In [21]:
db_session.query(SS_Data).all()[0].to_dict()

{'Name': 'Christian McCaffrey',
 'Team': 'CAR',
 'Opponent': 'NO',
 'Position': 'RB',
 'Projection': 27.1773,
 'Actual': 'undefined',
 'Price': 10000,
 'Value': 2.71773,
 'proj_own': 31.8,
 'dk_point': 27.1773,
 'dk_25_percentile': 18.9,
 'dk_50_percentile': 25.85,
 'dk_75_percentile': 33.9,
 'dk_85_percentile': 38.4,
 'dk_95_percentile': 46.105,
 'dk_99_percentile': 58.602,
 'fd_points': 21.9361,
 'fd_25_percentile': 15.5,
 'fd_50_percentile': 21.1,
 'fd_75_percentile': 27.325,
 'fd_85_percentile': 30.815,
 'fd_95_percentile': 37.405,
 'fd_99_percentile': 50.001,
 'fdraft_points': 27.1773,
 'fdraft_25_percentile': 18.9,
 'fdraft_50_percentile': 25.85,
 'fdraft_75_percentile': 33.9,
 'fdraft_85_percentile': 38.4,
 'fdraft_95_percentile': 46.105,
 'fdraft_99_percentile': 58.602,
 'yahoo_points': 21.9361,
 'yahoo_25_percentile': 15.5,
 'yahoo_50_percentile': 21.1,
 'yahoo_75_percentile': 27.325,
 'yahoo_85_percentile': 30.815,
 'yahoo_95_percentile': 37.405,
 'yahoo_99_percentile': 50.00